In [1]:
Steps to Perform for MFCC Transfer Learning
Step 1: Load the files and convert to mfcc

SyntaxError: invalid syntax (<ipython-input-1-050b52373b23>, line 1)

In [4]:
import numpy as np
import librosa as lp
import subprocess
import os
from IPython.core.debugger import set_trace
from sklearn.model_selection import train_test_split
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical


# This function is to convert mp3 to wav currently in disuse
def convert_audio_to_wav(filename):
    subprocess.call(['ffmpeg', '-i', filename,
                   str(filename.split(".mp3")[0])+".wav"])
    
#Function to convert mp3 audio to mfcc shapes    
def convert_to_mfcc(filename, max_length):
   # convert_audio_to_wav(filename)
    wave, sr = lp.load(filename, mono=True, sr=None)
    wave = wave[::3]
    mfcc = lp.feature.mfcc(wave, sr=16000)
     # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_length > mfcc.shape[1]):
        pad_width = max_length - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_length]
    
    return mfcc


def create_training_data(pathname_training):
    #set_trace()
    mfcc_array=[]
    subdirs=[x[1] for x in os.walk(pathname_training)][0]
    for subdir in subdirs:
        subdir_path=pathname_training+"\\" + subdir
        files2=[x[2] for x in os.walk(subdir_path)][0]
        print(files2)
        for file_name in files2:
            file_path=subdir_path+"\\"+file_name
            mfcc_of_file=convert_to_mfcc(file_path,11)
            mfcc_array.append(mfcc_of_file)
        np.save(subdir,mfcc_array)
        mfcc_array=[]

#create_training_data("C:\Users\Ashok\Documents\GitHub\SpeakerRecognition\Training")  

def get_mfcc_from_file(location_of_mfcc):
    mfcc_files=os.listdir(location_of_mfcc)
    return mfcc_files

#get_mfcc_from_file( os.getcwd()+"\\mfcc_files")

def get_train_test(ratio_of_split, random_state=42):
    # Get available labels
    mfcc_labels= get_mfcc_from_file( os.getcwd()+"\\mfcc_files")

    # Getting first arrays
    X = np.load(os.getcwd()+"\\mfcc_files\\"+mfcc_labels[0])
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(mfcc_labels[1:]):
        x = np.load(os.getcwd()+"\\mfcc_files\\"+label)
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    return train_test_split(X, y, test_size= (1 - ratio_of_split), random_state=random_state, shuffle=True)

X_train, X_test, y_train, y_test = get_train_test(0.6)
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(20, 11, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(14, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(X_train, y_train_hot, batch_size=100, epochs=200, verbose=1, validation_data=(X_test, y_test_hot))





TypeError: add() got an unexpected keyword argument 'W_regularizer'